### Galaxias de Gadget

In [1]:
import matplotlib.pyplot as plt
import h5py
import rotation_mio as rot
import numpy as np
import time_conversion as time

h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter
vector2 = ('A','B','C')
vector = ('00','01','03')

#path = 'home/ornela/SimCLUES/'
path0 = '/home/omarioni/'

atime = np.loadtxt(path0 + 'redshift_outputs.txt')
aexp = atime[:,2]
aexp = aexp[::-1]

path1 = '/mnt/sersic2/omarioni/'

path2 = '/mnt/is2/alejandro/ornella/'

snap = h5py.File(path2 + 'outputs_1/snap_496.h5py', 'r')

for i in range(0,3):
    
    
    output_file = h5py.File(path1 + 'vale/gadget/galaxy_'+str('%s'%vector2[i])+'.h5py', 'a')
    
    
    cm   = snap['subhalo_0'+ str('%s' %vector[i])+ '/Center'][()]
    r200 = snap['subhalo_0'+ str('%s' %vector[i])+ '/R200'][()]

    pstr  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Str/Coordinates'][()]
    mstr  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Str/Masses'][()]
    vstr  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Str/Velocities'][()]
    IDs   = snap['subhalo_0'+ str('%s' %vector[i]) + '/Str/Ids'][()]
    tform = snap['subhalo_0'+ str('%s'%vector[i])  + '/Str/FormationTime'][()]

    pdrk  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Drk/Coordinates'][()]
    mdrk  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Drk/Masses'][()]
    vdrk  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Drk/Velocities'][()]
    IDd   = snap['subhalo_0'+ str('%s' %vector[i]) + '/Drk/Ids'][()]
    
    pgas  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Gas/Coordinates'][()]
    mgas  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Gas/Masses'][()]
    vgas  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Gas/Velocities'][()]
    IDg   = snap['subhalo_0'+ str('%s' %vector[i]) + '/Gas/Ids'][()]
    
    z = a0/aexp[0] - 1.
    Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
    
    
    t_form = time.conv2(tform, h, omega_lambda, omega_matter)
    
    
#---aca paso las coordenadas respecto al centro de la galaxia------
    xstr = (pstr[:,0]-cm[0])*aexp[0]/h
    ystr = (pstr[:,1]-cm[1])*aexp[0]/h
    zstr = (pstr[:,2]-cm[2])*aexp[0]/h
    r = np.sqrt(xstr**2+ystr**2+zstr**2)
    
    xdrk = (pdrk[:,0]-cm[0])*aexp[0]/h
    ydrk = (pdrk[:,1]-cm[1])*aexp[0]/h
    zdrk = (pdrk[:,2]-cm[2])*aexp[0]/h
    
    xgas = (pgas[:,0]-cm[0])*aexp[0]/h
    ygas = (pgas[:,1]-cm[1])*aexp[0]/h
    zgas = (pgas[:,2]-cm[2])*aexp[0]/h

    vx_str = vstr[:,0] *np.sqrt(aexp[0]) + Ht *xstr/1000.
    vy_str = vstr[:,1] *np.sqrt(aexp[0]) + Ht *ystr/1000.
    vz_str = vstr[:,2] *np.sqrt(aexp[0]) + Ht *zstr/1000.
    
    vx_drk = vdrk[:,0] *np.sqrt(aexp[0]) + Ht *xdrk/1000.
    vy_drk = vdrk[:,1] *np.sqrt(aexp[0]) + Ht *ydrk/1000.
    vz_drk = vdrk[:,2] *np.sqrt(aexp[0]) + Ht *zdrk/1000.
    
    vx_gas = vgas[:,0] *np.sqrt(aexp[0]) + Ht *xgas/1000.
    vy_gas = vgas[:,1] *np.sqrt(aexp[0]) + Ht *ygas/1000.
    vz_gas = vgas[:,2] *np.sqrt(aexp[0]) + Ht *zgas/1000.

#----------------------masas----------------------------
    mstr = mstr*1e10/h
    mdrk = mdrk*1e10/h
    mgas = mgas*1e10/h

    r200 = r200*aexp[0]/h
    rgal=0.15*r200
      
    veloc,=np.where(r<3*aexp[0])

#----------componentes de la velocidad del centro de masa------------
    vxcm = sum(mstr[veloc]*vx_str[veloc])/sum(mstr[veloc])
    vycm = sum(mstr[veloc]*vy_str[veloc])/sum(mstr[veloc])
    vzcm = sum(mstr[veloc]*vz_str[veloc])/sum(mstr[veloc])

#----- velocidades de las estrellas respecto del centro de masa de la galaxia---------
    vx_str = vx_str - vxcm
    vy_str = vy_str - vycm
    vz_str = vz_str - vzcm
    
    vx_drk = vx_drk - vxcm
    vy_drk = vy_drk - vycm
    vz_drk = vz_drk - vzcm
    
    vx_gas = vx_gas - vxcm
    vy_gas = vy_gas - vycm
    vz_gas = vz_gas - vzcm

    
    e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vx_str,vy_str,vz_str,3*aexp[0])

##posiciones de partículas que se quiere graficar
##como lo de arriba me da los versores hago las posiciones con esto 

    xstr_n = e1x*xstr + e1y*ystr + e1z*zstr
    ystr_n = e2x*xstr + e2y*ystr + e2z*zstr
    zstr_n = e3x*xstr + e3y*ystr + e3z*zstr
    vxstr_n = e1x*vx_str + e1y*vy_str + e1z*vz_str
    vystr_n = e2x*vx_str + e2y*vy_str + e2z*vz_str
    vzstr_n = e3x*vx_str + e3y*vy_str + e3z*vz_str
    
    rstr = np.sqrt(xstr_n**2 + ystr_n**2 + zstr_n**2)
    
    limit_str = np.where(rstr < r200)
    
    pos_str=np.ndarray([np.size(rstr[limit_str]),3])
    pos_str[:,0]=xstr_n[limit_str]
    pos_str[:,1]=ystr_n[limit_str]
    pos_str[:,2]=zstr_n[limit_str]
    
    vel_str=np.ndarray([np.size(rstr[limit_str]),3])
    vel_str[:,0]=vxstr_n[limit_str]
    vel_str[:,1]=vystr_n[limit_str]
    vel_str[:,2]=vzstr_n[limit_str]
    
#--------------------------------------------------------------
    xdrk_n = e1x*xdrk + e1y*ydrk + e1z*zdrk
    ydrk_n = e2x*xdrk + e2y*ydrk + e2z*zdrk
    zdrk_n = e3x*xdrk + e3y*ydrk + e3z*zdrk
    vxdrk_n = e1x*vx_drk + e1y*vy_drk + e1z*vz_drk
    vydrk_n = e2x*vx_drk + e2y*vy_drk + e2z*vz_drk
    vzdrk_n = e3x*vx_drk + e3y*vy_drk + e3z*vz_drk
    
    rdrk = np.sqrt(xdrk_n**2 + ydrk_n**2 + zdrk_n**2)
    
    limit_drk = np.where(rdrk < r200)
    
    pos_drk=np.ndarray([np.size(rdrk[limit_drk]),3])
    pos_drk[:,0]=xdrk_n[limit_drk]
    pos_drk[:,1]=ydrk_n[limit_drk]
    pos_drk[:,2]=zdrk_n[limit_drk]
    
    vel_drk=np.ndarray([np.size(rdrk[limit_drk]),3])
    vel_drk[:,0]=vxdrk_n[limit_drk]
    vel_drk[:,1]=vydrk_n[limit_drk]
    vel_drk[:,2]=vzdrk_n[limit_drk]
    
#--------------------------------------------------------------
    xgas_n = e1x*xgas + e1y*ygas + e1z*zgas
    ygas_n = e2x*xgas + e2y*ygas + e2z*zgas
    zgas_n = e3x*xgas + e3y*ygas + e3z*zgas
    vxgas_n = e1x*vx_gas + e1y*vy_gas + e1z*vz_gas
    vygas_n = e2x*vx_gas + e2y*vy_gas + e2z*vz_gas
    vzgas_n = e3x*vx_gas + e3y*vy_gas + e3z*vz_gas
    
    rgas = np.sqrt(xgas_n**2 + ygas_n**2 + zgas_n**2)
    
    limit_gas = np.where(rgas < r200)
    
    pos_gas=np.ndarray([np.size(rgas[limit_gas]),3])
    pos_gas[:,0]=xgas_n[limit_gas]
    pos_gas[:,1]=ygas_n[limit_gas]
    pos_gas[:,2]=zgas_n[limit_gas]
    
    vel_gas=np.ndarray([np.size(rgas[limit_gas]),3])
    vel_gas[:,0]=vxgas_n[limit_gas]
    vel_gas[:,1]=vygas_n[limit_gas]
    vel_gas[:,2]=vzgas_n[limit_gas]
    
    
    output_file.create_dataset('/R200', data=r200)
    
    output_file.create_dataset('/Str/Coordinates',   data=pos_str, dtype=np.float32)
    output_file.create_dataset('/Str/Velocities',    data=vel_str, dtype=np.float32)
    output_file.create_dataset('/Str/Masses',        data=mstr[limit_str], dtype=np.float32)
    output_file.create_dataset('/Str/Ids',           data=IDs[limit_str])
    output_file.create_dataset('/Str/FormationTime', data=t_form[limit_str], dtype=np.float32)

    output_file.create_dataset('/Gas/Coordinates',   data=pos_gas, dtype=np.float32)
    output_file.create_dataset('/Gas/Velocities',    data=vel_gas, dtype=np.float32)
    output_file.create_dataset('/Gas/Masses',        data=mgas[limit_gas], dtype=np.float32)
    output_file.create_dataset('/Gas/Ids',           data=IDg[limit_gas])
#       output_file.create_dataset('/subhalo_'+str('%03d'% j)+'/Gas/Density',        data=rhogas[kgas], dtype=np.float32)
#       output_file.create_dataset('/subhalo_'+str('%03d'% j)+'/Gas/InternalEnergy', data=tempgas[kgas], dtype=np.float32)


    output_file.create_dataset('/Drk/Coordinates',   data=pos_drk, dtype=np.float32)
    output_file.create_dataset('/Drk/Velocities',    data=vel_drk, dtype=np.float32)
    output_file.create_dataset('/Drk/Masses',        data=mdrk[limit_drk], dtype=np.float32)
    output_file.create_dataset('/Drk/Ids',           data=IDd[limit_drk])

    #if( (snap.npart[4] > 0) & (np.size(kstr) > 0) ):

    

/home/omarioni/miniconda2/lib/python2.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [13]:
import h5py
snap = h5py.File('/mnt/sersic2/omarioni/vale/gadget/galaxy_A.h5py', 'r')
def printname(name):
    print name
snap.visit(printname)

Drk
Drk/Coordinates
Drk/Ids
Drk/Masses
Drk/Velocities
Gas
Gas/Coordinates
Gas/Ids
Gas/Masses
Gas/Velocities
R200
Str
Str/Coordinates
Str/FormationTime
Str/Ids
Str/Masses
Str/Velocities


In [14]:
gal = h5py.File('/mnt/sersic2/omarioni/vale/gadget/galaxy_A.h5py', 'r') 
    
r200 = gal['/Str/Masses'].value

In [15]:
print r200


[30229.883 30229.883 30229.883 ... 30229.883 30229.883 30229.883]


## Galaxias Gasoline

In [26]:
vx_str = vstr[0]

print vx_str

[371.27271402 411.44939701 -34.49917669]


In [24]:
print mstr[veloc]
vx_str[veloc]

[14348.54936595 14348.54936595 14348.54936595 ... 20191.72299061
 20191.72299061 20191.72299061]


IndexError: index 3 is out of bounds for axis 1 with size 3

In [29]:
'''aca guardo los snaps en el formato h5py'''
import matplotlib.pyplot as plt
import pynbody
import rotation_mio as rot
import h5py
import numpy as np

G = 4.299e-6
#a0 = 1
#r200 = (189.977454324, 173.656484976, 137.617865798) este es el viejo r200 no le des bolilla
vector=(1,2,3)
vector2 = ('A','B','C')

    
#     output_file = h5py.File('../outputs2/snap_'+str('%s'%snap[j])+'.h5py', 'a')
   
s=pynbody.load('/srv/cosmdatc/clues/B64_WM3_186592/LG/GAS_SFR/4096_Gasoline/1024/WMAP3.CLUES.HR.001024')
h = s.halos()

time = s.properties['time'].in_units('Gyr')
aexp = s.properties['a']
hh = s.properties['h']
omega_matter = s.properties['omegaM0']
omega_lambda = s.properties['omegaL0']

for i in range(0,3):
    
    output_file = h5py.File('/z/omarioni/vale/galaxy_'+str('%s'%vector2[i])+'.h5py', 'a')
               
    h1 = h[vector[i]]
               
    r200 = h1.properties['Rvir']

    cen_pot = pynbody.analysis.halo.center(h1,mode='pot',retcen=True).in_units('kpc')
               
               
    pstr  = h1.star['pos'].in_units('kpc')
    mstr  = h1.star['mass'].in_units('Msol')
    vstr  = h1.star['vel'].in_units('km s**-1')
    IDs   = h1.star['iord']
    tform = h1.star['tform'].in_units('Gyr')
               

    pdrk  = h1.dm['pos'].in_units('kpc')
    mdrk  = h1.dm['mass'].in_units('Msol')
    vdrk  = h1.dm['vel'].in_units('km s**-1')
    IDd   = h1.dm['iord']
               
    
    pgas  = h1.gas['pos'].in_units('kpc') 
    mgas  = h1.gas['mass'].in_units('Msol')
    vgas  = h1.gas['vel'].in_units('km s**-1')
    IDg   = h1.gas['iord']

#pos---------------------
    xstr = pstr[:,0]-cen_pot[0]
    ystr = pstr[:,1]-cen_pot[1]
    zstr = pstr[:,2]-cen_pot[2]
    r = np.sqrt(xstr**2 + ystr**2 + zstr**2)
    
    xdrk = pdrk[:,0]-cen_pot[0]
    ydrk = pdrk[:,1]-cen_pot[1]
    zdrk = pdrk[:,2]-cen_pot[2]
               
    xgas = pgas[:,0]-cen_pot[0]
    ygas = pgas[:,1]-cen_pot[1]
    zgas = pgas[:,2]-cen_pot[2]

#vel----------------------------
    vx_str = vstr[:,0]
    vy_str = vstr[:,1]
    vz_str = vstr[:,2]
               
    vx_gas = vgas[:,0]
    vy_gas = vgas[:,1]
    vz_gas = vgas[:,2]
               
    vx_drk = vdrk[:,0]
    vy_drk = vdrk[:,1]
    vz_drk = vdrk[:,2]
               
    veloc,=np.where(r<3*aexp)

#----------componentes de la velocidad del centro de masa------------
    vxcm = sum(mstr[veloc]*vx_str[veloc])/sum(mstr[veloc])
    vycm = sum(mstr[veloc]*vy_str[veloc])/sum(mstr[veloc])
    vzcm = sum(mstr[veloc]*vz_str[veloc])/sum(mstr[veloc])

#----- velocidades de las estrellas respecto del centro de masa de la galaxia---------
    vx_str = vx_str - vxcm
    vy_str = vy_str - vycm
    vz_str = vz_str - vzcm
    
    vx_drk = vx_drk - vxcm
    vy_drk = vy_drk - vycm
    vz_drk = vz_drk - vzcm
    
    vx_gas = vx_gas - vxcm
    vy_gas = vy_gas - vycm
    vz_gas = vz_gas - vzcm

    
    e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vx_str,vy_str,vz_str,3*aexp)

##posiciones de partículas que se quiere graficar
##como lo de arriba me da los versores hago las posiciones con esto 

    xstr_n = e1x*xstr + e1y*ystr + e1z*zstr
    ystr_n = e2x*xstr + e2y*ystr + e2z*zstr
    zstr_n = e3x*xstr + e3y*ystr + e3z*zstr
    vxstr_n = e1x*vx_str + e1y*vy_str + e1z*vz_str
    vystr_n = e2x*vx_str + e2y*vy_str + e2z*vz_str
    vzstr_n = e3x*vx_str + e3y*vy_str + e3z*vz_str
    
    rstr = np.sqrt(xstr_n**2 + ystr_n**2 + zstr_n**2)
    
    limit_str = np.where(rstr < r200)
    
    pos_str=np.ndarray([np.size(rstr[limit_str]),3])
    pos_str[:,0]=xstr_n[limit_str]
    pos_str[:,1]=ystr_n[limit_str]
    pos_str[:,2]=zstr_n[limit_str]
    
    vel_str=np.ndarray([np.size(rstr[limit_str]),3])
    vel_str[:,0]=vxstr_n[limit_str]
    vel_str[:,1]=vystr_n[limit_str]
    vel_str[:,2]=vzstr_n[limit_str]
    
#--------------------------------------------------------------
    xdrk_n = e1x*xdrk + e1y*ydrk + e1z*zdrk
    ydrk_n = e2x*xdrk + e2y*ydrk + e2z*zdrk
    zdrk_n = e3x*xdrk + e3y*ydrk + e3z*zdrk
    vxdrk_n = e1x*vx_drk + e1y*vy_drk + e1z*vz_drk
    vydrk_n = e2x*vx_drk + e2y*vy_drk + e2z*vz_drk
    vzdrk_n = e3x*vx_drk + e3y*vy_drk + e3z*vz_drk
    
    rdrk = np.sqrt(xdrk_n**2 + ydrk_n**2 + zdrk_n**2)
    
    limit_drk = np.where(rdrk < r200)
    
    pos_drk=np.ndarray([np.size(rdrk[limit_drk]),3])
    pos_drk[:,0]=xdrk_n[limit_drk]
    pos_drk[:,1]=ydrk_n[limit_drk]
    pos_drk[:,2]=zdrk_n[limit_drk]
    
    vel_drk=np.ndarray([np.size(rdrk[limit_drk]),3])
    vel_drk[:,0]=vxdrk_n[limit_drk]
    vel_drk[:,1]=vydrk_n[limit_drk]
    vel_drk[:,2]=vzdrk_n[limit_drk]
    
#--------------------------------------------------------------
    xgas_n = e1x*xgas + e1y*ygas + e1z*zgas
    ygas_n = e2x*xgas + e2y*ygas + e2z*zgas
    zgas_n = e3x*xgas + e3y*ygas + e3z*zgas
    vxgas_n = e1x*vx_gas + e1y*vy_gas + e1z*vz_gas
    vygas_n = e2x*vx_gas + e2y*vy_gas + e2z*vz_gas
    vzgas_n = e3x*vx_gas + e3y*vy_gas + e3z*vz_gas
    
    rgas = np.sqrt(xgas_n**2 + ygas_n**2 + zgas_n**2)
    
    limit_gas = np.where(rgas < r200)
    
    pos_gas=np.ndarray([np.size(rgas[limit_gas]),3])
    pos_gas[:,0]=xgas_n[limit_gas]
    pos_gas[:,1]=ygas_n[limit_gas]
    pos_gas[:,2]=zgas_n[limit_gas]
    
    vel_gas=np.ndarray([np.size(rgas[limit_gas]),3])
    vel_gas[:,0]=vxgas_n[limit_gas]
    vel_gas[:,1]=vygas_n[limit_gas]
    vel_gas[:,2]=vzgas_n[limit_gas]
               
               
               
               
               
    output_file.create_dataset('/R200', data=r200)

    output_file.create_dataset('/Str/Coordinates',   data=pos_str, dtype=np.float32)
    output_file.create_dataset('/Str/Velocities',    data=vel_str, dtype=np.float32)
    output_file.create_dataset('/Str/Masses',        data=mstr[limit_str], dtype=np.float32)
    output_file.create_dataset('/Str/Ids',           data=IDs[limit_str])
    output_file.create_dataset('/Str/FormationTime', data=tform[limit_str], dtype=np.float32)

    output_file.create_dataset('/Gas/Coordinates',   data=pos_gas, dtype=np.float32)
    output_file.create_dataset('/Gas/Velocities',    data=vel_gas, dtype=np.float32)
    output_file.create_dataset('/Gas/Masses',        data=mgas[limit_gas], dtype=np.float32)
    output_file.create_dataset('/Gas/Ids',           data=IDg[limit_gas])
#       output_file.create_dataset('/subhalo_'+str('%03d'% j)+'/Gas/Density',        data=rhogas[kgas], dtype=np.float32)
#       output_file.create_dataset('/subhalo_'+str('%03d'% j)+'/Gas/InternalEnergy', data=tempgas[kgas], dtype=np.float32)


    output_file.create_dataset('/Drk/Coordinates',   data=pos_drk, dtype=np.float32)
    output_file.create_dataset('/Drk/Velocities',    data=vel_drk, dtype=np.float32)
    output_file.create_dataset('/Drk/Masses',        data=mdrk[limit_drk], dtype=np.float32)
    output_file.create_dataset('/Drk/Ids',           data=IDd[limit_drk])

        #if( (snap.npart[4] > 0) & (np.size(kstr) > 0) ):


In [30]:
import h5py
snap = h5py.File('/z/omarioni/vale/galaxy_A.h5py', 'r')
def printname(name):
    print name
snap.visit(printname)

Drk
Drk/Coordinates
Drk/Ids
Drk/Masses
Drk/Velocities
Gas
Gas/Coordinates
Gas/Ids
Gas/Masses
Gas/Velocities
R200
Str
Str/Coordinates
Str/FormationTime
Str/Ids
Str/Masses
Str/Velocities


In [33]:
r200 = snap['Str/Coordinates'][()]

In [36]:
print np.shape(r200)


(975780, 3)
